In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
from keras import backend as K
from keras.optimizers import Adam

from data_prepare import read_images_from_file, read_labels_from_file, H_IMAGE, W_IMAGE

from random import sample

import matplotlib.pyplot as plt

%matplotlib inline

Using TensorFlow backend.


# Pre-train

In [3]:
NUM_TRAIN_FILES = 11

In [4]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(H_IMAGE, W_IMAGE, 3))
base_model.output.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'concat_14:0' shape=(?, 3, 3, 2048) dtype=float32>>

In [5]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

model = Model(input=base_model.input, output=predictions)

for layer in base_model.layers:
    layer.trainable = False

In [6]:
Y_train_pcnt = []

for part_i in range(NUM_TRAIN_FILES):        
    Y_train = read_labels_from_file('./data/train-labels.part{}.ubyte'.format(part_i))
    Y_train = np.asarray([[(i == Y) for i in range(1, 5)] for Y in Y_train])
    Y_train_pcnt.append(np.sum(Y_train, axis=0) / float(Y_train.shape[0]))

print np.asarray(Y_train_pcnt, dtype=float)

[[ 0.28133333  0.28133333  0.15066667  0.28666667]
 [ 0.37933333  0.37933333  0.072       0.16933333]
 [ 0.39933333  0.39933333  0.02933333  0.172     ]
 [ 0.34933333  0.34933333  0.12533333  0.176     ]
 [ 0.37866667  0.37866667  0.052       0.19066667]
 [ 0.26733333  0.26733333  0.164       0.30133333]
 [ 0.27933333  0.27933333  0.15066667  0.29066667]
 [ 0.30666667  0.30666667  0.12533333  0.26133333]
 [ 0.36066667  0.36066667  0.116       0.16266667]
 [ 0.29866667  0.29866667  0.14        0.26266667]
 [ 0.384       0.384       0.03        0.202     ]]


In [7]:
model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
nb_epoch = 3

In [8]:
for epoch in range(nb_epoch):
    print "-" * 20 + " Epoch {}/{} ".format(epoch+1, nb_epoch) + "-" * 20
    
    for part_i in sample(range(NUM_TRAIN_FILES), NUM_TRAIN_FILES):
        X_train = read_images_from_file('./data/train-images.part{}.ubyte'.format(part_i))
        
        Y_train = read_labels_from_file('./data/train-labels.part{}.ubyte'.format(part_i))
        Y_train = np.asarray([[(i == Y) for i in range(1, 5)] for Y in Y_train])
        
        model.fit(X_train, Y_train, nb_epoch=1, batch_size=50)

-------------------- Epoch 1/3 --------------------
Epoch 1/1
3000/3000 [==============================] - 279s - loss: 1.2266   
Epoch 1/1
3000/3000 [==============================] - 262s - loss: 1.0440   
Epoch 1/1
3000/3000 [==============================] - 276s - loss: 0.9651   
Epoch 1/1
3000/3000 [==============================] - 263s - loss: 0.8507   
Epoch 1/1
2000/2000 [==============================] - 179s - loss: 0.8399   
Epoch 1/1
3000/3000 [==============================] - 262s - loss: 0.8874   
Epoch 1/1
3000/3000 [==============================] - 270s - loss: 0.8318   
Epoch 1/1
3000/3000 [==============================] - 260s - loss: 0.8414   
Epoch 1/1
3000/3000 [==============================] - 267s - loss: 0.7505   
Epoch 1/1
3000/3000 [==============================] - 260s - loss: 0.9515   
Epoch 1/1
3000/3000 [==============================] - 269s - loss: 0.8762   
-------------------- Epoch 2/3 --------------------
Epoch 1/1
3000/3000 [=================

In [9]:
f_arch = open('./models/model_arch1.json', 'w')
f_arch.write(model.to_json())
model.save_weights('./models/model_weights1.h5')
f_arch.close()

In [10]:
model.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
nb_epoch = 2

In [11]:
for epoch in range(nb_epoch):
    print "-" * 20 + " Epoch {}/{} ".format(epoch+1, nb_epoch) + "-" * 20
    
    for part_i in sample(range(NUM_TRAIN_FILES), NUM_TRAIN_FILES):
        X_train = read_images_from_file('./data/train-images.part{}.ubyte'.format(part_i))
        
        Y_train = read_labels_from_file('./data/train-labels.part{}.ubyte'.format(part_i))
        Y_train = np.asarray([[(i == Y) for i in range(1, 5)] for Y in Y_train])
        
        model.fit(X_train, Y_train, nb_epoch=1, batch_size=50)

-------------------- Epoch 1/2 --------------------
Epoch 1/1
2000/2000 [==============================] - 187s - loss: 0.6685 - acc: 0.7190   
Epoch 1/1
3000/3000 [==============================] - 286s - loss: 0.6893 - acc: 0.7187   
Epoch 1/1
3000/3000 [==============================] - 284s - loss: 0.6042 - acc: 0.7623   
Epoch 1/1
3000/3000 [==============================] - 286s - loss: 0.8054 - acc: 0.6617   
Epoch 1/1
3000/3000 [==============================] - 291s - loss: 0.7597 - acc: 0.6743   
Epoch 1/1
3000/3000 [==============================] - 288s - loss: 0.7208 - acc: 0.6923   
Epoch 1/1
3000/3000 [==============================] - 280s - loss: 0.7231 - acc: 0.7103   
Epoch 1/1
3000/3000 [==============================] - 293s - loss: 0.7253 - acc: 0.6963   
Epoch 1/1
3000/3000 [==============================] - 282s - loss: 0.6400 - acc: 0.7480   
Epoch 1/1
3000/3000 [==============================] - 287s - loss: 0.6813 - acc: 0.7233   
Epoch 1/1
3000/3000 [=======

In [13]:
f_arch = open('./models/model2_arch.json', 'w')
f_arch.write(model.to_json())
model.save_weights('./models/model2_weights.h5')
f_arch.close()

In [37]:
%reset_selective -f X_train

In [38]:
%reset_selective -f Y_train

# Submission 1

In [16]:
NUM_TEST_FILES = 5

In [17]:
Y_labels = []

In [21]:
for part_i in range(NUM_TEST_FILES):
    X_test = read_images_from_file('./data/test-images.part{}.ubyte'.format(part_i))
        
    Y_test = model.predict(X_test, verbose=1, batch_size=50)
    Y_test = np.argmax(Y_test, axis=1).tolist()
    
    Y_labels.extend(Y_test)

1999/1999 [==============================] - 185s   


In [22]:
submission = pd.read_csv('./data/sample_submission4.csv')

In [23]:
submission['label'] = Y_labels
submission['label'] += 1
submission.to_csv('./submission_1.csv', sep=',', index=False)

In [32]:
%reset_selective -f X_test

# Train

In [24]:
for i, layer in enumerate(base_model.layers):
    print i, layer.name

0 input_1
1 convolution2d_1
2 batchnormalization_1
3 convolution2d_2
4 batchnormalization_2
5 convolution2d_3
6 batchnormalization_3
7 maxpooling2d_1
8 convolution2d_4
9 batchnormalization_4
10 convolution2d_5
11 batchnormalization_5
12 maxpooling2d_2
13 convolution2d_9
14 batchnormalization_9
15 convolution2d_7
16 convolution2d_10
17 batchnormalization_7
18 batchnormalization_10
19 averagepooling2d_1
20 convolution2d_6
21 convolution2d_8
22 convolution2d_11
23 convolution2d_12
24 batchnormalization_6
25 batchnormalization_8
26 batchnormalization_11
27 batchnormalization_12
28 mixed0
29 convolution2d_16
30 batchnormalization_16
31 convolution2d_14
32 convolution2d_17
33 batchnormalization_14
34 batchnormalization_17
35 averagepooling2d_2
36 convolution2d_13
37 convolution2d_15
38 convolution2d_18
39 convolution2d_19
40 batchnormalization_13
41 batchnormalization_15
42 batchnormalization_18
43 batchnormalization_19
44 mixed1
45 convolution2d_23
46 batchnormalization_23
47 convolution2d_

In [25]:
for layer in model.layers[:172]:
    layer.trainable = False

for layer in model.layers[172:]:
    layer.trainable = True

In [35]:
model.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
nb_epoch = 10

In [39]:
for epoch in range(nb_epoch):
    print "-" * 20 + " Epoch {}/{} ".format(epoch+1, nb_epoch) + "-" * 20
    
    for part_i in sample(range(NUM_TRAIN_FILES), NUM_TRAIN_FILES):
        X_train = read_images_from_file('./data/train-images.part{}.ubyte'.format(part_i))
        
        Y_train = read_labels_from_file('./data/train-labels.part{}.ubyte'.format(part_i))
        Y_train = np.asarray([[(i == Y) for i in range(1, 5)] for Y in Y_train])
        
        model.fit(X_train, Y_train, nb_epoch=1, validation_split=0.0625, batch_size=50)

-------------------- Epoch 1/10 --------------------
Train on 2812 samples, validate on 188 samples
Epoch 1/1
2812/2812 [==============================] - 363s - loss: 0.6696 - acc: 0.7258 - val_loss: 0.9780 - val_acc: 0.8032
Train on 2812 samples, validate on 188 samples
Epoch 1/1
2812/2812 [==============================] - 384s - loss: 0.7798 - acc: 0.6839 - val_loss: 0.6049 - val_acc: 0.7660
Train on 2812 samples, validate on 188 samples
Epoch 1/1
2812/2812 [==============================] - 350s - loss: 0.6794 - acc: 0.7219 - val_loss: 0.5426 - val_acc: 0.8085
Train on 2812 samples, validate on 188 samples
Epoch 1/1
2812/2812 [==============================] - 381s - loss: 0.6075 - acc: 0.7521 - val_loss: 0.5209 - val_acc: 0.7553
Train on 2812 samples, validate on 188 samples
Epoch 1/1
2812/2812 [==============================] - 349s - loss: 0.6453 - acc: 0.7344 - val_loss: 0.5325 - val_acc: 0.7979
Train on 2812 samples, validate on 188 samples
Epoch 1/1
2812/2812 [==============

In [40]:
f_arch = open('./models/model3_arch.json', 'w')
f_arch.write(model.to_json())
model.save_weights('./models/model3_weights.h5')
f_arch.close()

In [41]:
%reset_selective -f X_train

In [42]:
%reset_selective -f Y_train

# Submission 2

In [43]:
NUM_TEST_FILES = 5

In [44]:
Y_labels = []

In [45]:
for part_i in range(NUM_TEST_FILES):
    X_test = read_images_from_file('./data/test-images.part{}.ubyte'.format(part_i))
        
    Y_test = model.predict(X_test, verbose=1, batch_size=50)
    Y_test = np.argmax(Y_test, axis=1).tolist()
    
    Y_labels.extend(Y_test)

1999/1999 [==============================] - 172s   


In [46]:
submission = pd.read_csv('./data/sample_submission4.csv')

In [47]:
submission['label'] = Y_labels
submission['label'] += 1
submission.to_csv('./submission_2.csv', sep=',', index=False)

In [48]:
%reset_selective -f X_test